In [2]:
import pickle
import os
import numpy as np
import tensorflow as tf

import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.slim as slim
import tensorflow.nn as nn

print("Imported")

Imported


In [3]:
input_dir = 'data/pickles/'

In [8]:
label_2_num_mapping = {}
num_2_label_ar= []
data = {"x":[],"y":[]}

label_num = 0

for label in os.listdir(input_dir):
    if label == '.DS_Store' or label == '.ipynb_checkpoints':
        continue
    p_file = input_dir + label
    label_2_num_mapping[label]=label_num
    num_2_label_ar.append(label)

    with open(p_file, 'rb') as f:
        label_examples  = pickle.load(f)
        # data dim = num_examples * time * frequency
        label_ar = [np.swapaxes(np.array(example),0,1) for example in label_examples]
        data['x'].extend(label_ar)
        y_labels = [label_num for i in range(0,len(label_examples))]
        data['y'].extend(y_labels)
   
    label_num+=1
    
print(data['x'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
def sample_batch(n, X, Y):
    """
    takes input and returns padded sample
    n= num_samples
    X = input_featurs_list
    Y = label_list
    """
    label_ids = np.random.randint(low = 0,high = len(X), size=n)
    frequency = len(X[0][0])
    sampled_X = [X[label_id] for label_id in label_ids]
    sampled_y = [Y[label_id] for label_id in label_ids]
    padded_X = []
    
    max_batch_len = max([len(x) for x in sampled_X])
    for x in sampled_X:
        padding_time_count = max_batch_len-len(x)
        if padding_time_count!=0:
            x_padded = np.zeros(shape = (max_batch_len,frequency))
            x_padded[:x.shape[0],:x.shape[1]] = x
            padded_X.append(x_padded)
        else:
            padded_X.append(x)
        
    return np.asarray(padded_X),np.asarray(sampled_y)

In [5]:
class Word_Predictor_RNN():
    def __init__(self, batch_size=1, input_freq=20,hidden_size=256,linear_output_size = 128, n_categories=30):
            super().__init__()
            
            self.forward_lstm_cells = rnn.BasicLSTMCell(hidden_size) 
            self.backward_lstm_cells = rnn.BasicLSTMCell(hidden_size) 
            
            self.init_state_fw = self.forward_lstm_cells.zero_state(batch_size, tf.float32)
            self.init_state_bw = self.backward_lstm_cells.zero_state(batch_size, tf.float32)
            
            print(self.init_state_fw)
            print(self.init_state_bw)

            input = tf.placeholder(tf.float32, [1,hidden_size], name='output')
            print(input)
            self.linear1 = slim.fully_connected(input, linear_output_size, activation_fn=nn.relu6)

            input = tf.placeholder(tf.float32, [1,linear_output_size], name='categorize')
            print(input)
            self.linear2 = slim.fully_connected(input, n_categories, activation_fn=None)
    
            
    def forward(self, x):
        # [length, batch_size, embedding_size]
        # x = tf.reshape(x, [-1])
        print(x)
        
        (output_fw, output_bw), last_state = tf.nn.bidirectional_dynamic_rnn(self.forward_lstm_cells, 
                                                                             self.backward_lstm_cells,
                                                                             x,
                                                                             initial_state_fw=self.init_state_fw,
                                                                             initial_state_bw=self.init_state_bw,
                                                                             dtype=tf.float64)
        
        return output_fw, output_bw

In [9]:
n_categories = len(label_2_num_mapping)
batch_size = 1
input_freq = 20
hidden_size=256
n_epochs = 10
n_iters = 50

linear_output_size = 128
n_categories=30

In [10]:
model = Word_Predictor_RNN(batch_size=batch_size, input_freq=input_freq, hidden_size=hidden_size, n_categories=n_categories)

losses = np.zeros(n_epochs) # For plotting


# Build Graph

with tf.Session() as sess:

    # Train Network
    
    for epoch in range(n_epochs):

        for iter in range(n_iters):
            _inputs,_labels = sample_batch(50,data['x'],data['y'])
            print(_inputs.shape)
            
            x = tf.reshape(_inputs, [50,-1])
            print(np.shape(x))

            # forward 
            output_fw, output_bw = model.forward(_inputs)
            output = tf.concat([output_fw, output_bw], axis=2)
            
            activations = tf.contrib.layers.fully_connected(output[:-1], linear_output_size, activation_fn=nn.relu6)
            predictions = tf.contrib.layers.fully_connected(activations, n_categories, activation_fn=None)
            
            loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=predictions,labels =_labels)
            
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

            
            





LSTMStateTuple(c=<tf.Tensor 'BasicLSTMCellZeroState/zeros:0' shape=(1, 256) dtype=float32>, h=<tf.Tensor 'BasicLSTMCellZeroState/zeros_1:0' shape=(1, 256) dtype=float32>)
LSTMStateTuple(c=<tf.Tensor 'BasicLSTMCellZeroState_1/zeros:0' shape=(1, 256) dtype=float32>, h=<tf.Tensor 'BasicLSTMCellZeroState_1/zeros_1:0' shape=(1, 256) dtype=float32>)
Tensor("output:0", shape=(1, 256), dtype=float32)
Tensor("categorize:0", shape=(1, 128), dtype=float32)
(50, 44, 20)
(50, 880)
[[[-6.01242729e+02  8.74686402e-02  8.74488810e-02 ...  8.55785340e-02
    8.53497652e-02  8.51081431e-02]
  [-6.01091672e+02  3.01079408e-01  3.01011394e-01 ...  2.94573395e-01
    2.93785941e-01  2.92954243e-01]
  [-6.01263818e+02  5.76463189e-02  5.76332965e-02 ...  5.64006420e-02
    5.62498716e-02  5.60906302e-02]
  ...
  [-4.18022586e+02  8.04941638e+01  5.25504058e+00 ...  3.32880003e+00
    4.42409500e-01  4.57419570e+00]
  [-4.43219369e+02  7.41262559e+01  9.58687756e+00 ...  4.11330715e+00
    2.65916405e-01  3.

TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [float64, float32] that don't all match.